HardAttention, hidden state is picked randomly

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

import os
import pickle
import numpy as np
import math
from torchsummary import summary

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path=''

In [ ]:
a = [2,34,5,76,8,94,3,2]
a[:1]

[2]

In [ ]:
class DeapS2SDatasetClassification(torch.utils.data.Dataset):
    
    def __init__(self, path):

        _, _, filenames = next(os.walk(path))
        filenames = sorted(filenames)
        all_data = []
        all_label = []
        for dat in filenames:
            temp = pickle.load(open(os.path.join(path,dat), 'rb'), encoding='latin1')
            all_data.append(temp['data'])
            all_label.append(temp['labels'][:,1:2]) #For valance & #For arousal [:,1:2]
        
        self.data = np.vstack(all_data)
        self.label = np.vstack(all_label)
        del temp, all_data, all_label

    def __len__(self):
        return self.data.shape[0]

   
    def __getitem__(self, idx):
        single_data = self.data[idx]
        single_label = (self.label[idx] > 5).astype(float)
        
        batch = {
            'data': torch.Tensor(single_data),
            'label': torch.Tensor(single_label)
        }

        return batch

In [ ]:
def classification_report(pred,actual,best_class_weights):
    acc = round(best_class_weights[0]*(accuracy_score(np.vstack(pred).flatten(), np.vstack(actual).flatten())),2)
    precision = round(best_class_weights[1]*precision_score(np.vstack(pred).flatten(), np.vstack(actual).flatten()),2)
    recall = round(best_class_weights[1]*recall_score(np.vstack(pred).flatten(), np.vstack(actual).flatten()),2)
    f1score = round(best_class_weights[1]*f1_score(np.vstack(pred).flatten(), np.vstack(actual).flatten()),2)
    return acc,precision,recall,f1score

In [ ]:
dataset = DeapS2SDatasetClassification(path+'data_preprocessed_python')

torch.manual_seed(1)
indices = torch.randperm(len(dataset)).tolist()
train_ind = int(0.7 * len(dataset))
train_set = torch.utils.data.Subset(dataset, indices[:train_ind])
val_set = torch.utils.data.Subset(dataset, indices[train_ind:])
del dataset

print(len(train_set))
print(len(val_set))

train_loader = torch.utils.data.DataLoader(train_set, batch_size=16, shuffle=True, pin_memory=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=16, shuffle=False, pin_memory=True)

896
384


In [ ]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.hidden_size = hidden_size
        self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
#initializing random hidden size
        self.v = nn.Parameter(torch.rand(hidden_size))
        

    def forward(self, hidden, encoder_outputs):
      #Length of encoder o/p
        timestep = encoder_outputs.size(0)

      #we have repeate the length of hidden unit
        h = hidden.repeat(timestep, 1, 1).transpose(0, 1)

      #transposing
        encoder_outputs = encoder_outputs.transpose(0, 1)  
  
  #cat (enc_o/p,hidden_states)
        temp = torch.cat([h, encoder_outputs], dim=2)
  #applying linear layer, relu activation fun to calculate attention weights
        energy = F.relu(self.attn(temp))
  #reshaping
        energy = energy.transpose(1, 2) 
  #Since hidden states are to be picked random, so we are applying v
        v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1) 
  #multiplying with energies 
        energy = torch.bmm(v, energy)
        attn_energies = energy.squeeze(1)
  #applying siftmax_fun
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_size, embed_size,
                 n_layers=1, dropout=0.5):
        super(Encoder, self).__init__()

        self.embed_size = embed_size
        self.lstm = nn.LSTM(input_size, embed_size, n_layers,
                          dropout=dropout, bidirectional=True)

    def forward(self, x):

        output, (hn, cn) = self.lstm(x)
        output = (output[:, :, :self.embed_size] +
                   output[:, :, self.embed_size:])
        return output, hn



In [ ]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size,
                 dropout=0.2):
        super(Decoder, self).__init__()

        self.hidden_size = hidden_size
        self.output_size = output_size

        self.dropout = nn.Dropout(dropout, inplace=True)
        self.attention = Attention(hidden_size)

        self.fc = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size * 2, output_size)
        self.sig = nn.Sigmoid()

    def forward(self, last_hidden, encoder_outputs):
        attn_weights = self.attention(last_hidden[-1], encoder_outputs)
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))  
        context = context.transpose(0, 1)  
        output = self.fc(last_hidden.view(-1, 2*self.hidden_size))
        context = context.squeeze(0)
        output = self.out(torch.cat([output, context], 1))
        return self.sig(output), attn_weights

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src):

        encoder_output, hidden = self.encoder(src) 
        output, attn_weights = self.decoder(hidden, encoder_output)

        return output

In [ ]:
dataiter = iter(train_loader)
data = dataiter.next()
images, labels = data['data'],data['label']
print(images.shape)
print(labels.shape)

torch.Size([16, 40, 8064])
torch.Size([16, 1])


In [ ]:
enc = Encoder(40, 128, 1).cuda()
dec = Decoder(128, 1).cuda()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
s2s = Seq2Seq(enc, dec).to(device)
loss_fn = nn.BCELoss()
lr = 0.01
best_class_weights=[1.5,1.35]
optimizer = torch.optim.AdamW(s2s.parameters(), lr=lr)

C:\Users\Shashank\anaconda3\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [ ]:
print(s2s)

Seq2Seq(
  (encoder): Encoder(
    (lstm): LSTM(40, 128, dropout=0.5, bidirectional=True)
  )
  (decoder): Decoder(
    (dropout): Dropout(p=0.2, inplace=True)
    (attention): Attention(
      (attn): Linear(in_features=256, out_features=128, bias=True)
    )
    (fc): Linear(in_features=256, out_features=128, bias=True)
    (out): Linear(in_features=256, out_features=1, bias=True)
    (sig): Sigmoid()
  )
)


In [ ]:
for epoch in range(15):
    s2s.train()
    train_loss = 0

    for i, batch in enumerate(train_loader):
        data = batch['data'].permute(2, 0, 1).cuda()
        label = batch['label'].cuda()
        optimizer.zero_grad()
        output = s2s(data)
        loss = loss_fn(output, label)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    s2s.eval()
    val_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(val_loader):

            data = batch['data'].permute(2, 0, 1).cuda()
            label = batch['label'].cuda()
            output = s2s(data)
            loss = loss_fn(output, label)
            val_loss += loss.item()

    print('Epoch : {} train_loss : {} val_loss : {}'.format(epoch, train_loss/len(train_loader), val_loss/len(val_loader)))

Epoch : 0 train_loss : 0.762541518679687 val_loss : 0.7091310347119967
Epoch : 1 train_loss : 0.7232447160141808 val_loss : 0.7173792943358421
Epoch : 2 train_loss : 0.7096917863403048 val_loss : 0.8363188107808431
Epoch : 3 train_loss : 0.7585396372846195 val_loss : 0.7413972616195679
Epoch : 4 train_loss : 0.7412136356745448 val_loss : 0.7448438107967377
Epoch : 5 train_loss : 0.7777547389268875 val_loss : 0.7963129306832949
Epoch : 6 train_loss : 0.7458632795938424 val_loss : 0.7834710851311684
Epoch : 7 train_loss : 0.7462451191885131 val_loss : 0.7398806611696879
Epoch : 8 train_loss : 0.7618906998208591 val_loss : 0.7884946515162786
Epoch : 9 train_loss : 0.7775557919272355 val_loss : 0.7447085852424303
Epoch : 10 train_loss : 0.7839349476354462 val_loss : 0.7577720309297243
Epoch : 11 train_loss : 0.7609124934034688 val_loss : 0.7416837761799494
Epoch : 12 train_loss : 0.7758487564112458 val_loss : 0.7789229154586792
Epoch : 13 train_loss : 0.7489275389483997 val_loss : 0.795376

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score


fin_targets = []
fin_outputs = []
with torch.no_grad():
    for i, batch in enumerate(train_loader):

        data = batch['data'].permute(2, 0, 1).cuda()
        label = batch['label']
        output = s2s(data)
        fin_targets.append(label.numpy())
        fin_outputs.append(np.asarray((output.cpu().detach().numpy()>0.5), dtype=np.int))
        
acc,precision,recall,f1score = classification_report(fin_outputs,fin_targets,best_class_weights)

print('Accuracy : {}'.format(acc))
print('Precision: {}'.format(precision))
print('Recall: {}'.format(recall))
print('F1score: {}'.format(f1score))

Accuracy : 0.87
Precision: 1.09
Recall: 0.81
F1score: 0.93
